In [12]:
import cv2

cap = cv2.VideoCapture("/Users/kevynkrancenblum/Desktop/Data Science/Final Project/LipNet-main/data/s1/bbaf4p.mpg")

while True:
    ret, frame = cap.read()
    print(frame.shape)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 360, 3)
(288, 

AttributeError: 'NoneType' object has no attribute 'shape'

In [13]:
import mediapipe as mp
import tensorflow as tf
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [14]:
def detection_preprocessing(image, h_max=360):
    h, w, _ = image.shape
    if h > h_max:
        ratio = h_max / h
        w_ = int(w * ratio)
        image = cv2.resize(image, (w_,h_max))
    return image

def resize_face(face):
    x = tf.expand_dims(tf.convert_to_tensor(face), axis=2)
    return tf.image.resize(x, (48,48))

def recognition_preprocessing(faces):
    x = tf.convert_to_tensor([resize_face(f) for f in faces])
    return x

In [31]:
esultat=[]
def inference(image):
    H, W, _ = image.shape
    
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_image)

    if results.detections:
        faces = []
        pos = []
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            mp_drawing.draw_detection(image, detection)

            x = int(box.xmin * W)
            y = int(box.ymin * H)
            w = int(box.width * W)
            h = int(box.height * H)

            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(x + w, W)
            y2 = min(y + h, H)

            face = image[y1:y2,x1:x2]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            faces.append(face)
            pos.append((x1, y1, x2, y2))
            
        return faces, pos

cap = cv2.VideoCapture("/Users/kevynkrancenblum/Desktop/Data Science/Final Project/Facial_emotion_recognition/ModelVideos/Noa_angry.MOV")

while True:
    ret, frame = cap.read()
    try:
        faces = inference(frame)
        if len(faces) > 0:
            faces = recognition_preprocessing(faces)
            for face in faces:
                cv2.imshow('frame', face)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
    except:
        cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break